### 003 - MANTENDO LINHA HISTÓRICA DOS DADOS - SILVER ZONE

In [0]:
# -------------------------------------------------------------------------------------------------------------------------------------------- #
# Importando bibliotecas necessarias
# -------------------------------------------------------------------------------------------------------------------------------------------- #

from __future__ import print_function 
import re
import sys 
from pandas import DataFrame, concat
import requests
import xml.etree.ElementTree as ET
import logging
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import NullType
from pyspark.sql.functions import *
import pandas as pd


# -------------------------------------------------------------------------------------------------------------------------------------------- #
# Realizando leitura dos arquivos + Interpretando como temp view
# -------------------------------------------------------------------------------------------------------------------------------------------- #
df004a  = spark.read.option("charset", "ISO-8859-1").parquet("/mnt/bronze_zone/tb_t_caixaTem/")
print('Leitura completa dos dados!')


# -------------------------------------------------------------------------------------------------------------------------------------------- #
# RReplicação dos dados com registro do current timestamp
# -------------------------------------------------------------------------------------------------------------------------------------------- #
df004b = df004a \
            .withColumn('ts_gravado_em', current_timestamp()) \
            .coalesce(1)


# -------------------------------------------------------------------------------------------------------------------------------------------- #
# Realiza gravação dos dados em bronze zone no datalake
# PARTIÇÃO: ano / mes (a partir do timestamo do registro)
# MODO: overwrite 
# DIRETÓRIO DATALAKE: /mnt/bronze_zone/tb_e_caixaTem
# PROPRIETARIO: Gabriel Oliveira / Gabriel Carvlho
# -------------------------------------------------------------------------------------------------------------------------------------------- #
(
    df004b
    .write
    .partitionBy("pt_ano","pt_mes")
    .mode("overwrite")
    .parquet("/mnt/silver_zone/tb_t_caixaTem")
)
print('''
# ****************************************************************************
            Gravação dos dados - Parte 3 - Silver - já concluído!
# ****************************************************************************
''')

Leitura completa dos dados!

# ****************************************************************************
 Gravação dos dados - Parte 3 - Silver - já concluído!
# ****************************************************************************

In [0]:
df004b.limit(10).toPandas()

Out[6]:

,nu_id,ds_title,ds_reviews,nu_sum_votes_reviews,ds_count_votes_reviews,nu_rating_review,dt_date,hr_review,ts_review,nu_version_app,pt_ano,pt_mes,ts_gravado_em
0,8933400063,OTIMO,OTIMO,0,0,5,2022-08-01,02:49:57,2022-08-01T02:49:57-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
1,8933433956,NAO CONSIGO VER O SALDO,O APLICATIVO PEDE PARA REALIZAR ATUALIZACAO CA...,0,0,1,2022-08-01,03:03:56,2022-08-01T03:03:56-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
2,8933883783,CAIXA TEM,EXCELENTE,0,0,5,2022-08-01,05:54:43,2022-08-01T05:54:43-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
3,8934019142,ERRO,ESTA DANDO ERRO AO CONSULTAR O SALDO,0,0,1,2022-08-01,06:39:46,2022-08-01T06:39:46-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
4,8934180317,MAGDA,EU GOSTEI BASTANTE ME EXPLICA MUITO,0,0,5,2022-08-01,07:32:12,2022-08-01T07:32:12-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
5,8934246521,NAO TENHO ACESSO AO MEU SALDO,PEDE PARA CONFIRMAR IDENTIDADE EU VOU E CONFIR...,0,0,1,2022-08-01,07:54:10,2022-08-01T07:54:10-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
6,8934382159,SALDO,ESTOU O DIA TODO TENTANDO SACAR MEU SEGURO DES...,0,0,1,2022-08-01,08:40:13,2022-08-01T08:40:13-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
7,8934498122,PESSIMO,"E O PIOR APP DE BANCO QUE EXISTE, TRAVA TODA H...",0,0,1,2022-08-01,09:20:44,2022-08-01T09:20:44-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
8,8934513689,ERRO AO FINALIZAR PROCESSO DE CARTAO DE CREDITO,ERRO CONSTANTE AO FINALIZAR O PROCESSO DE PEDI...,0,0,1,2022-08-01,09:26:22,2022-08-01T09:26:22-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726
9,8934626566,PESSIMO! HORRIVEL! PESSIMO 1000 VEZES,ESTER636373,0,0,1,2022-08-01,10:08:41,2022-08-01T10:08:41-07:00,1.61.0,2022,8,2022-08-29 02:56:47.726


### Testes: Documentação dos testes da aplicação pyspark;

O objetivo dessa tarefa é realizar testes aplicados ao dataset obtido na ingestão dos dados, seguindo o padrão exigido pelo quadro abaixo.

In [0]:
df004b.createOrReplaceTempView("vw_caixaTem")

In [0]:
%sql

with base as ( SELECT DISTINCT * FROM vw_caixaTem )

SELECT count(*) as total_registros, substr(dt_date,1,7) as data FROM base  GROUP BY substr(dt_date,1,7);


-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
--  Exibindo o total de registros por ano-mes
-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

total_registros,data
451,2022-08
49,2022-07


In [0]:
%sql

SELECT count(*) as total, nu_sum_votes_reviews, ds_count_votes_reviews  FROM vw_caixaTem GROUP BY nu_sum_votes_reviews, ds_count_votes_reviews ORDER BY nu_sum_votes_reviews, ds_count_votes_reviews DESC

-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
--  Count total por número de soma de votos e count total de votos por pessoal
-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

total,nu_sum_votes_reviews,ds_count_votes_reviews
1,0,1
475,0,0
21,1,1
3,2,2


In [0]:
%sql

SELECT count(*) as total, nu_rating_review  FROM vw_caixaTem GROUP BY nu_rating_review ORDER BY nu_rating_review DESC

-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
--  Count total por avaliação
-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

total,nu_rating_review
44,5
7,4
16,3
20,2
413,1


In [0]:
%sql

SELECT count(*) as total, nu_version_app  FROM vw_caixaTem GROUP BY nu_version_app ORDER BY nu_version_app DESC

-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
--  Count total por versão do app
-- * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

total,nu_version_app
170,1.63.0
249,1.62.0
81,1.61.0


In [0]:
%sql

(SELECT ds_title, ds_reviews, '202206' as ano FROM vw_caixaTem WHERE substr(dt_date,1,7) = '2022-06' limit 3)
UNION ALL
(SELECT ds_title, ds_reviews, '202207' as ano FROM vw_caixaTem WHERE substr(dt_date,1,7) = '2022-07' limit 3)


ds_title,ds_reviews,ano
CAIXA TEM,"DEPOIS DESSA ULTIMA ATUALIZACAO O APP FICOU UM LIXO, PEDE PRA ATUALIZAR OS DADOS O TEMPO TODO E NAO FUNCIONA NADA, ARRUMEM ISSO.",202207
PIOR APP,PESSIMO,202207
APLICATIVO APRESENTA MUITOS ERROS,"QUANDO VOCE SELECIONA ALGUMA OPCAO DA ERRO, NAO CONSIGO VER O SALDO DA CONTA, APLICATIVO E MUITO RUIM",202207


In [0]:
df_evd = spark.sql(''' SELECT * FROM vw_caixaTem''')

df_evd.limit(5).toPandas()


Out[8]:

,nu_id,ds_title,ds_reviews,nu_sum_votes_reviews,ds_count_votes_reviews,nu_rating_review,dt_date,hr_review,ts_review,nu_version_app,pt_ano,pt_mes,ts_gravado_em
0,8933400063,OTIMO,OTIMO,0,0,5,2022-08-01,02:49:57,2022-08-01T02:49:57-07:00,1.61.0,2022,8,2022-08-29 03:01:03.260
1,8933433956,NAO CONSIGO VER O SALDO,O APLICATIVO PEDE PARA REALIZAR ATUALIZACAO CA...,0,0,1,2022-08-01,03:03:56,2022-08-01T03:03:56-07:00,1.61.0,2022,8,2022-08-29 03:01:03.260
2,8933883783,CAIXA TEM,EXCELENTE,0,0,5,2022-08-01,05:54:43,2022-08-01T05:54:43-07:00,1.61.0,2022,8,2022-08-29 03:01:03.260
3,8934019142,ERRO,ESTA DANDO ERRO AO CONSULTAR O SALDO,0,0,1,2022-08-01,06:39:46,2022-08-01T06:39:46-07:00,1.61.0,2022,8,2022-08-29 03:01:03.260
4,8934180317,MAGDA,EU GOSTEI BASTANTE ME EXPLICA MUITO,0,0,5,2022-08-01,07:32:12,2022-08-01T07:32:12-07:00,1.61.0,2022,8,2022-08-29 03:01:03.260
